# 1.获取小说资源

自行下载epub文件，在此转换为txt格式并合并为一个文件，已有txt格式则忽略这步

In [1]:
# from ebooklib import epub
# from bs4 import BeautifulSoup
# import os

# def extract_text_from_epub(file_path):
#     book = epub.read_epub(file_path)
#     text_content = []
#     for item in book.items:
#         if item.media_type == 'application/xhtml+xml':
#             soup = BeautifulSoup(item.content,'html.parser')
#             text_content.append(soup.get_text())
#     return "\n".join(text_content)

# def merge_epubs_to_txt(epub_files, output_txt_file):
#     with open(output_txt_file, "w", encoding="utf-8") as f:
#         for file_name in os.listdir(epub_files):
#             if file_name.endswith('.epub'):
#                 file_path = os.path.join(epub_files,file_name)
#                 print(f"Porcess {file_name}...")
#                 text = extract_text_from_epub(file_path)
#                 f.write(text + "\n\n")


# # 使用示例
# epub_files = "./novel/败犬1-6-epub"
# output_txt_file = "TooManyLosingHeroines.txt"   
# merge_epubs_to_txt(epub_files, output_txt_file)

# 2.小说切分

In [2]:
# 选择你要处理的小说，接下来的保存路径都将使用小说名
novel_name = "Honglou-full"
input_txt_name = f"./novel/{novel_name}.txt"
save_folder = f"./extract/{novel_name}"

# target_role: 要抽取的人物名称
# 支持空字符串(默认前三个)或者 list[str] ，如果出错默认保存第一个
target_role = ["黛玉", "宝玉", "宝钗"]

In [3]:
import os

if not os.path.exists(save_folder):
    os.makedirs(save_folder)
else:
    print('注意，文件夹',save_folder,'已经存在')

raw_text = open(input_txt_name, encoding='utf-8').read()

# 将文本按章节切分
chapters = []
chapter_contents = []

for line in raw_text.split('\n'):
    Flag = False
    if line.strip().startswith("～第"):
        # 遇到章节标题,将之前章节内容添加到结果列表

        head = line.strip()
        # print(head)
        head = head[:min(10,len(head))]
        if head.find('败',1)>0:
            # print(head)
            Flag = True

    if Flag:
        if chapter_contents:
            chapters.append('\n'.join(chapter_contents))
            chapter_contents = []
        # 记录当前章节标题
        # chapters.append(line)
    else:
        # 累积章节内容
        chapter_contents.append(line)

# 添加最后一个章节内容
if chapter_contents:
    chapters.append('\n'.join(chapter_contents))
# 检查结果
print(len(chapters))
for i,ch in enumerate(chapters):
    print(i,ch[:10])

1
0 第一回  甄士隐梦幻


In [4]:
# @title  定义divide函数，用来切分超长文本
def divide_str(s, sep=["\n", ".", "。"]):
    mid_len = len(s) // 2  # 中心点位置
    best_sep_pos = len(s) + 1  # 最接近中心点的分隔符位置
    best_sep = None  # 最接近中心点的分隔符
    for curr_sep in sep:
        sep_pos = s.rfind(curr_sep, 0, mid_len)  # 从中心点往左找分隔符
        if sep_pos > 0 and abs(sep_pos - mid_len) < abs(best_sep_pos - mid_len):
            best_sep_pos = sep_pos
            best_sep = curr_sep
    if not best_sep:  # 没有找到分隔符
        return s, ""
    return s[: best_sep_pos + 1], s[best_sep_pos + 1 :]


def strong_divide(s):
    left, right = divide_str(s)

    if right != "":
        return left, right

    whole_sep = [
        "\n",
        ".",
        "，",
        "、",
        ";",
        ",",
        "；",
        "：",
        "！",
        "？",
        "(",
        ")",
        "”",
        "“",
        "’",
        "‘",
        "[",
        "]",
        "{",
        "}",
        "<",
        ">",
        "/",
        """''', '|', '-', '=', '+', '*', '%', \
               '$', """  #''', '@', '&', '^', '_', '`', '~',\
        "·",
        "…",
    ]
    left, right = divide_str(s, sep=whole_sep)

    if right != "":
        return left, right

    mid_len = len(s) // 2
    return s[:mid_len], s[mid_len:]

In [5]:
# @title 以1500 token为限，切分chunk，输出总chunk数量
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")
max_token_len = 1500
chunk_text = []


for chapter in chapters:

    split_text = chapter.split("\n")

    curr_len = 0
    curr_chunk = ""

    tmp = []

    for line in split_text:
        line_len = len(enc.encode(line))

        if line_len <= max_token_len - 5:
            tmp.append(line)
        else:
            # print('divide line with length = ', line_len)
            path = [line]
            tmp_res = []

            while path:
                my_str = path.pop()
                left, right = strong_divide(my_str)

                len_left = len(enc.encode(left))
                len_right = len(enc.encode(right))

                if len_left > max_token_len - 15:
                    path.append(left)
                else:
                    tmp_res.append(left)

                if len_right > max_token_len - 15:
                    path.append(right)
                else:
                    tmp_res.append(right)

            for line in tmp_res:
                tmp.append(line)

    split_text = tmp

    for line in split_text:
        line_len = len(enc.encode(line))

        if line_len > max_token_len:
            print("warning line_len = ", line_len)

        if curr_len + line_len <= max_token_len:
            curr_chunk += line
            curr_chunk += "\n"
            curr_len += line_len
            curr_len += 1
        else:
            chunk_text.append(curr_chunk)
            curr_chunk = line
            curr_len = line_len

    if curr_chunk:
        chunk_text.append(curr_chunk)

    # break

print(len(chunk_text))
print(chunk_text[20])

959
黛玉一一的都答应着。只见一个丫鬟来回：“老太太那里传晚饭了！”王夫人忙携黛玉从后房门由后廊往西，出了角门，是一条南北宽夹道。南边是倒座三间小小抱厦厅，北边立着一个粉油大影壁，后有一半大门，小小一所房室。王夫人笑指向黛玉道：“这是你凤姐姐的屋子，回来你好往这里找她来，少什么东西，你只管和她说就是了。”这院门上也有四五个才总角的小厮，都垂手侍立。王夫人遂携黛玉穿过一个东西穿堂，便是贾母的后院了。于是，进入后房门，已有多人在此伺候，见王夫人来了，方安设桌椅。贾珠之妻李氏捧饭，熙凤安箸，王夫人进羹。贾母正面榻上独坐，两边四张空椅，熙凤忙拉了黛玉在左边第一张椅上坐了。黛玉十分推让。贾母笑道：“你舅母和嫂子们不在这里吃饭。你是客，原应如此坐的。”黛玉方告了座，坐了。贾母命王夫人坐了。迎春姊妹三个告了座，方上来。迎春便坐右手第一，探春左第二，惜春右第二。旁边丫鬟执着拂尘、漱盂、巾帕。李、凤二人立于案旁布让。外间伺候之媳妇丫鬟虽多，却连一声咳嗽不闻。寂然饭毕，各有丫鬟用小茶盘捧上茶来。当日林如海教女以惜福养身，云饭后务待饭粒咽尽，过一时再吃茶，方不伤脾胃。今黛玉见了这里许多事情不合家中之式，不得不随的，少不得一一的改过来，因而接了茶。早有人又捧过漱盂来，黛玉也照样漱了口。然后盥手毕，又捧上茶来，方是吃的茶。贾母便说：“你们去罢，让我们自在说话儿。”王夫人听了，忙起身，又说了两句闲话，方引李、凤二人去了。贾母因问黛玉念何书。黛玉道：“只刚念了《四书》。”黛玉又问姊妹们读何书。贾母道：“读的是什么书，不过是认得两个字，不是睁眼的瞎子罢了！”



# 3.重组小说
    将小说重组为人物对话和剧情摘要

In [6]:
# !pip install langchain_community kor
# !pip install -U langchain langchain-openai

In [7]:
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
import os

# 设置你自己的LLM key与model
key = os.getenv("OPENAI_API_KEY", "your_api_key")
model = "gpt-4o-mini"

llm = ChatOpenAI(model_name=model, temperature=0)

In [8]:
# @title 仅仅抽取dialogue的部分

# @title 抽取role,dialogue,action

schema = Object(
    id="script",
    description='''Adapted from the novel into script,
    The main character,“温水”, refers to himself as “我” ''',
    attributes=[
        Text(
            id="role",
            description="The character who is speaking, use context to predict the name of the role.",
        ),
        Text(
            id="dialogue",
            description="The dialogue spoken by the characters in the sentence",
        ),
        Text(
            id ="action",
            descrtption = '''The actions performed by the characters in the text, A high-level summary of a character's behavior. action equals "对话" or "独白" or "动作", 
            equals "对话" if it's sentence in [], equals "独白" elif it's a psychological monologue not in [], equals "动作" else
            ''',
        ),
    ],
    examples=[
        (
            """``八奈见不知何时来到我身旁，用手肘轻轻地顶我。
「喔，烧盐同学好像喜欢绫野的样子。」
「是喔。真是意外的组合呢。」
「我和他们国中时同校，不过他们好像从国小就常常在一起。」
「所以说，就和青梅竹马差不多啰。」
青梅竹马……在八奈见眼中是这样吗？
对着无法理解的我，八奈见无奈地耸了耸肩。
「听我说，温水，女生可以分成两大类。如果不是青梅竹马，就是狐狸精。」
原来如此，真豪爽的分类法。八奈见用严厉的表情看着我。``""",
            [
                {"role":"八奈见","dialogue":"八奈见不知何时来到我身旁，用手肘轻轻地顶我","action":"动作",},
                {"role": "八奈见", "dialogue": "喔，烧盐同学好像喜欢绫野的样子","action":"对话",},
                {"role": "温水", "dialogue": "是喔。真是意外的组合呢。","action":"对话",},
                {
                    "role": "温水",
                    "dialogue": "我和他们国中时同校，不过他们好像从国小就常常在一起",
                    "action":"对话",
                },
                {
                    "role": "温水",
                    "dialogue": "青梅竹马……在八奈见眼中是这样吗？",
                    "action":"独白",
                },
                {"role": "八奈见", "dialogue": "所以说，就和青梅竹马差不多啰","action":"对话",},
                {
                    "role": "八奈见",
                    "dialogue": "听我说，温水，女生可以分成两大类。如果不是青梅竹马，就是狐狸精",
                    "action":"对话",
                },
                {
                    "role": "温水",
                    "dialogue": "原来如此，真豪爽的分类法",
                    "action":"独白",
                },
            ],
        )
    ],
    many=True,
)
chain = create_extraction_chain(llm, schema)

In [9]:
# test
test_chunk_id = 3
response = chain.invoke({"text":f"``{chunk_text[test_chunk_id]}"})
print(response["data"])

{'script': [{'role': '士隐', 'dialogue': '一日，炎夏永昼，士隐于书房闲坐，至手倦拋书，伏几少憩，不觉朦胧睡去', 'action': '独白'}, {'role': '道人', 'dialogue': '你携了这蠢物，意欲何往？', 'action': '对话'}, {'role': '僧', 'dialogue': '你放心，如今现有一段风流公案正该了结。这一干风流冤家，尚未投胎入世。趁此机会，就将此蠢物夹带于中，使他去经历经历', 'action': '对话'}, {'role': '道人', 'dialogue': '原来近日风流冤孽又将造劫历世去不成？但不知落于何方何处？', 'action': '对话'}, {'role': '僧', 'dialogue': '此事说来好笑，竟是千古未闻的罕事：只因西方灵河岸上三生石畔有绛珠草一株，时有赤瑕宫神瑛侍者，日以甘露灌溉，这绛珠草便得久延岁月', 'action': '对话'}, {'role': '僧', 'dialogue': '后来既受天地精华，复得雨露滋养，遂得脱却草胎木质，得换人形，仅修成个女体，终日游于离恨天外，饥则食蜜青果为膳，渴则饮灌愁海水为汤', 'action': '对话'}, {'role': '僧', 'dialogue': '只因尚未酬报灌溉之德，故其五内便郁结着一段缠绵不尽之意', 'action': '对话'}, {'role': '僧', 'dialogue': '恰近日这神瑛侍者凡心偶炽，乘此昌明太平朝世，意欲下凡造历幻缘，已在警幻仙子案前挂了号', 'action': '对话'}, {'role': '僧', 'dialogue': '警幻亦曾问及，灌溉之情未偿，趁此倒可了结的', 'action': '对话'}, {'role': '绛珠仙子', 'dialogue': '他是甘露之惠，我并无此水可还。他既下世为人，我也去下世为人，但把我一生所有的眼泪还他，也偿还得过他了', 'action': '对话'}, {'role': '道人', 'dialogue': '果是罕闻。实未闻有‘还泪’之说。想来这一段故事，比历来风月事故更加琐碎细腻了', 'action': '对话'}, {'role': '僧', 'dialogue': '历来几

In [10]:
system_prompt = """
Summarize the key points of the following text in a concise way, using bullet points.
"""

q_example = """###
Text:
洪七公、周伯通、郭靖、黄蓉四人乘了小船，向西驶往陆地。郭靖坐在船尾扳桨，黄蓉不住向周伯通详问骑鲨游海之事，周伯通兴起，当场就要设法捕捉鲨鱼，与黄蓉大玩一场。
郭靖见师父脸色不对，问道：“你老人家觉得怎样”洪七公不答，气喘连连，声息粗重。他被欧阳锋以“透骨打穴法”点中之后，穴道虽已解开，内伤却又加深了一层。黄蓉喂他服了几颗九花玉露丸，痛楚稍减，气喘仍是甚急。
老顽童不顾别人死活，仍是嚷着要下海捉鱼，黄蓉却已知不妥，向他连使眼色，要他安安静静的，别吵得洪七公心烦。周伯通并不理会，只闹个不休。黄蓉皱眉道：“你要捉鲨鱼，又没饵引得鱼来，吵些甚么”

Summarize in BULLET POINTS form:
"""

a_example = """
- 洪七公等四人乘船西行,洪七公因受内伤加重而气喘不止
- 周伯通要捉鲨鱼玩,被黄蓉阻止以免掀翻小船
"""

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [12]:
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=q_example),
    AIMessage(content=a_example),
]


new_input = f"""###
Text:
{chunk_text[test_chunk_id]}

Summarize in BULLET POINTS form:"""

messages.append(HumanMessage(content=new_input))

response = llm(messages)

print(response.content)

C:\Users\charles0618\AppData\Local\Temp\ipykernel_32624\254159305.py:16: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)


- 士隐在书房中打盹，梦见一僧一道谈论风流公案。
- 僧人提到要将“蠢物”夹带于风流冤家中，以便经历。
- 讨论中提到西方灵河岸上的绛珠草和神瑛侍者的故事。
- 神瑛侍者因凡心炽热，欲下凡造历幻缘，绛珠仙子愿意还泪以报恩。
- 僧人和道人计划去下世度脱风流冤家。
- 甄士隐听到谈话，询问“蠢物”是什么，僧人给他一块“通灵宝玉”。
- 僧人告知已到幻境，并带领道人过石牌坊，牌坊上写有“太虚幻境”。


In [13]:
import os
import json
from tqdm import tqdm

# 最贵的一步，这里只处理几个chunk作为测试
for i in tqdm(range(len(chunk_text))):
    save_name = os.path.join(save_folder, f"{i}_dialogue.txt")

    if not os.path.exists(save_name) or os.path.getsize(save_name) < 5:
        if os.path.exists(save_name):
            print("re-generate dialogue id = ", i)
        query_text = f"``{chunk_text[i]}``"
        # dialogue_response = chain.run( query_text )["data"]
        dialogue_response = chain.invoke({"text": f"``{chunk_text[i]}``"})["data"]
        with open(save_name, "w", encoding="utf-8") as f:
            if "script" not in dialogue_response:
                print('Error: response does not contain key "script"')
            else:
                for chat in dialogue_response["script"]:
                    json_str = json.dumps(chat, ensure_ascii=False)
                    f.write(json_str + "\n")

    save_name_sum = os.path.join(save_folder, f"{i}_sum.txt")

    if not os.path.exists(save_name_sum) or os.path.getsize(save_name_sum) < 5:
        if os.path.exists(save_name_sum):
            print("re-summarize id = ", i)
        # dealing with summarize
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=q_example),
            AIMessage(content=a_example),
        ]

        new_input = f"###\nText:\n{chunk_text[ i ]}\nSummarize in BULLET POINTS form:"

        messages.append(HumanMessage(content=new_input))

        summarize_response = llm(messages).content

        with open(save_name_sum, "w", encoding="utf-8") as f:
            f.write(summarize_response)

    raw_text_save_name = os.path.join(save_folder, f"{i}_raw.txt")

    if (
        not os.path.exists(raw_text_save_name)
        or os.path.getsize(raw_text_save_name) < 5
    ):
        with open(raw_text_save_name, "w", encoding="utf-8") as f:
            f.write(chunk_text[i])
    # TODO for test
    # if i > 3:
    #     break

100%|██████████| 959/959 [3:54:06<00:00, 14.65s/it]  


## 将得到的raw,dialogue,sum组合为规格的json

In [14]:
save_folder_path = f"./reorganized/{novel_name}"
# chunk所在文件夹，请以_raw结尾
folder_path = f"./extract/{novel_name}"
# 故事名字，默认为_raw之前的名字
story_name_en = novel_name

save_jsonl_path = f"{save_folder_path}/reorganized_{story_name_en}.jsonl"
save_txt_path = f"{save_folder_path}/reorganized_{story_name_en}.txt"


In [15]:
chunk_text = [""] * (
    sum([1 for file_name in os.listdir(folder_path) if file_name.endswith("_sum.txt")])
)

# 遍历文件夹中的文件
for file_name in os.listdir(folder_path):
    if file_name.endswith("_raw.txt"):
        file_path = os.path.join(folder_path, file_name)

        # 提取文件名中的 i
        i = int(file_name.split("_")[0])

        # 打开文件并读取内容
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read()

        # 将文件内容添加到列表中，按 i 的顺序插入到正确位置
        chunk_text[i] = text

print(len(chunk_text))

959


In [16]:

id = 2
dialoge_file = os.path.join(folder_path, f"{id}_dialogue.txt")
summarzie_file = os.path.join(folder_path, f"{id}_sum.txt")
raw_text = chunk_text[id]
chunk_sum = []
unique_chunk_sum = []
# 给定summarzie_file = os.path.join(save_folder, f"{id}_sum.txt")

# 先检查这个文件是否存在

# 然后使用utf-8编码打开，检查每一行，如果strip后，行首是'-'，则把后面的字符串append到一个list chunk_sum中

# 请用python为我实现
if os.path.exists(summarzie_file):
    with open(summarzie_file, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip().startswith("-"):
                chunk_sum.append(line.strip()[1:].strip())
if os.path.exists(dialoge_file):
    with open(dialoge_file, encoding="utf-8") as f:
        dialogues = []
        for line in f:
            dialogue = json.loads(line)
            dialogues.append(dialogue)

unique_dialogue = []
for item in dialogues:
    if item not in unique_dialogue:
        unique_dialogue.append(item)
dia_texts = [data["dialogue"] for data in unique_dialogue]
for item in chunk_sum:
    if item not in unique_chunk_sum:
        unique_chunk_sum.append(item)

chunk_sum = unique_chunk_sum
dialogues = unique_dialogue
print(dialogues)
print(dia_texts)
print(chunk_sum)

[{'role': '空空道人', 'dialogue': '空空道人听如此说，思忖半晌，将一这《石头记》再检阅一遍', 'action': '独白'}, {'role': '空空道人', 'dialogue': '因见上面虽有些指奸责佞、贬恶诛邪之语，亦非伤时骂世之旨', 'action': '独白'}, {'role': '空空道人', 'dialogue': '及至君仁臣良、父慈子孝，凡伦常所关之处，皆是称功颂德，眷眷无穷', 'action': '独白'}, {'role': '空空道人', 'dialogue': '实非别书之可比', 'action': '独白'}, {'role': '空空道人', 'dialogue': '虽其中大旨谈情，亦不过实录其事，又非假拟妄称，一味淫邀艳约，私订偷盟之可比', 'action': '独白'}, {'role': '空空道人', 'dialogue': '因毫不干涉时世，方从头至尾抄录回来，问世传奇', 'action': '独白'}, {'role': '空空道人', 'dialogue': '因空见色，由色生情，传情入色，自色悟空', 'action': '独白'}, {'role': '空空道人', 'dialogue': '空空道人遂易名为情僧，改《石头记》为《情僧录》', 'action': '独白'}, {'role': '空空道人', 'dialogue': '至?玉峰题曰《红楼梦》', 'action': '独白'}, {'role': '空空道人', 'dialogue': '东鲁孔梅溪则题曰《风月宝鉴》', 'action': '独白'}, {'role': '空空道人', 'dialogue': '后因曹雪芹于悼红轩中，披阅十载，增删五次，纂成目录，分出章回', 'action': '独白'}, {'role': '空空道人', 'dialogue': '则题曰《金陵十二钗》，并题一绝云：满纸荒唐言，一把辛酸泪！都云作者痴，谁解其中味？', 'action': '独白'}, {'role': '空空道人', 'dialogue': '至脂砚斋甲戌抄阅再评，仍用《石头记》', 'action': '独白'}, {'role': '空空道人', 'dialogue': '出则既明，且看石上是何故

In [17]:
# 给定长文本raw_text。

# 使用换行符\n或者。 来对这个字符串进行切割，忽略掉strip之后是空的子字符串

# 将每一段话的起点位置存储在一个list of int , starts中

# 将每一段话的结束位置存储在一个list of int , ends中

# 并且将每一个子字符串的存储在一个list of str, lines中
def divide_raw2lines(raw_text):
    previous_str = ""
    starts = []
    ends = []
    lines = []
    for i in range(len(raw_text)):
        previous_str += raw_text[i]
        if raw_text[i] in ("\n", "。"):
            strip_str = previous_str.strip(' "“”\r\n')
            if len(strip_str) > 0:
                lines.append(strip_str)
                starts.append(i - len(strip_str))
                ends.append(i)
            previous_str = ""
        else:
            pass
    return lines, starts, ends


lines, starts, ends = divide_raw2lines(raw_text)

print(len(lines))

18


In [18]:
# 已知if '\u4e00' <= char <= '\u9fa5': 可以判断一个char是否是中文字

# 我希望实现一个函数，这个函数的输入是两个list of string, 长度为M的query 和 长度为N的datas

# 输出是一个M*N的numpy float数组 recalls

# 先计算freqs[m][n] 表示query的第m句中的每一个中文字，是否在datas[n]中是否出现，如果出现，则freqs[m][n]加一

# 然后计算recalls[m][n]是freqs[m][n]除掉 query[m]中所有中文字的个数
import numpy as np

def compute_char_recall(query, datas):
    M = len(query)
    N = len(datas)

    freqs = np.zeros((M, N), dtype=int)

    for m in range(M):
        q_chars = set()
        for char in query[m]:
            if "\u4e00" <= char <= "\u9fa5":
                q_chars.add(char)

        for n in range(N):
            for char in q_chars:
                if char in datas[n]:
                    freqs[m][n] += 1

    query_chars_count = [
        len(set(char for char in sent if "\u4e00" <= char <= "\u9fa5"))
        for sent in query
    ]

    recalls = freqs / np.array(query_chars_count)[:, None]

    return recalls

In [19]:
# import copy
def summary2line(chunk_sum, lines):

    s = compute_char_recall(chunk_sum, lines)

    color_map = {}
    ans_Q = {}

    ans_div = {}

    flags = {}

    M = len(chunk_sum)
    N = len(lines)

    for n in range(0, N):
        if n == 0:
            ans_Q[(0, 0)] = s[0, 0]
            ans_div[(0, 0)] = []
        else:
            ans_Q[(0, n)] = ans_Q[(0, n - 1)] + s[0, n]
            ans_div[(0, n)] = []

    for m in range(1, M):
        ans_Q[(m, m)] = ans_Q[(m - 1, m - 1)] + s[m, m]
        ans_div[(m, m)] = ans_div[(m - 1, m - 1)].copy()
        ans_div[(m, m)].append(m)

    def find_Q(m, n):
        # print(m,n)

        if m < 0 or n < 0:
            print("error out bound", m, " ", n)
            return 0, []

        if (m, n) in ans_Q.keys():
            return ans_Q[(m, n)], ans_div[(m, n)]

        if (m, n) in color_map.keys():
            print("error repeated quest ", m, " ", n)
            return 0, []
        else:
            color_map[(m, n)] = 1

        current_div = []

        left, left_div = find_Q(m, n - 1)
        right, right_div = find_Q(m - 1, n - 1)

        if left > right:
            ans = left + s[m][n]
            flags[(m, n)] = False
            current_div = left_div

        else:
            ans = right + s[m][n]
            flags[(m, n)] = True
            current_div = right_div.copy()
            current_div.append(n - 1)

        # ans = max(  , ) + s[m][n]

        ans_Q[(m, n)] = ans
        ans_div[(m, n)] = current_div.copy()

        return ans, current_div

    # print(find_Q(0,5))
    # print(find_Q(M-1,N-1))

    score, divs = find_Q(M - 1, N - 1)
    divs.append(N - 1)

    return score, divs


score, divs = summary2line(chunk_sum, lines)
print(score, divs)

last = 0

# divs.append(N-1)

for i, div in enumerate(divs):
    print("\n###")
    print(chunk_sum[i])
    print("\n")

    for j in range(last, div):
        print(lines[j])
        last = div

3.991911197261741 [1, 2, 3, 4, 6, 8, 12, 17]

###
空空道人重新审视《石头记》，认为其内容并非单纯的时世批评。


空空道人听如此说，思忖半晌，将一这《石头记》再检阅一遍，因见上面虽有些指奸责佞、贬恶诛邪之语，亦非伤时骂世之旨；及至君仁臣良、父慈子孝，凡伦常所关之处，皆是称功颂德，眷眷无穷，实非别书之可比。

###
书中强调君仁臣良、父慈子孝等伦理道德，称功颂德。


虽其中大旨谈情，亦不过实录其事，又非假拟妄称，一味淫邀艳约，私订偷盟之可比。

###
书中虽有情感描写，但实录其事，非虚构淫乱之作。


因毫不干涉时世，方从头至尾抄录回来，问世传奇。

###
空空道人将书名改为《情僧录》，并由玉峰题为《红楼梦》。


因空见色，由色生情，传情入色，自色悟空，空空道人遂易名为情僧，改《石头记》为《情僧录》。

###
东鲁孔梅溪题为《风月宝鉴》，曹雪芹在悼红轩增删五次，最终定名《金陵十二钗》。


至?玉峰题曰《红楼梦》。
东鲁孔梅溪则题曰《风月宝鉴》。

###
曹雪芹的绝句表达了对作品的感慨与深意。


后因曹雪芹于悼红轩中，披阅十载，增删五次，纂成目录，分出章回，则题曰《金陵十二钗》，并题一绝云：
满纸荒唐言，一把辛酸泪！都云作者痴，谁解其中味？

###
书中提到的故事背景设定在东南的姑苏，阊门外的仁清巷。


至脂砚斋甲戌抄阅再评，仍用《石头记》。
出则既明，且看石上是何故事。
按那石上书云：
当日地陷东南，这东南一隅有处曰姑苏，有城曰阊门者，最是红尘中一二等富贵风流之地。

###
主要人物甄士隐，性情淡泊，热爱自然，膝下无儿，只有一女英莲。


这阊门外有个十里街，街内有个仁清巷，巷内有个古庙，因地方窄狭，人皆呼作葫芦庙。
庙旁住着一家乡宦，姓甄名费，字士隐。
嫡妻封氏，情性贤淑，深明礼义。
家中虽不甚富贵，然本地便也推他为望族了。
只因这甄士隐禀性恬淡，不以功名为念，每日只以观花修竹、酌酒吟诗为乐，倒是神仙一流人品。


In [20]:
def dialogue2line(dia_texts, lines):
    s_dialogue = compute_char_recall(dia_texts, lines)

    M, N = s_dialogue.shape
    if M == 0 or N == 0:
        return []
    dp = np.zeros((M, N))
    dp[0] = s_dialogue[0]
    prev_indices = np.zeros((M, N), dtype=int)
    for i in range(1, M):
        for j in range(N):
            max_prev_index = np.argmax(dp[i - 1])
            dp[i][j] = dp[i - 1][max_prev_index] + s_dialogue[i][j]
            prev_indices[i][j] = max_prev_index

    max_end_index = np.argmax(dp[-1])
    sequence = []
    for i in range(M - 1, -1, -1):
        sequence.append(max_end_index)
        max_end_index = prev_indices[i][max_end_index]
    sequence.reverse()

    return sequence


seq = dialogue2line(dia_texts, lines)
print(seq)

for i, id in enumerate(seq):
    print("\n###")
    print(dia_texts[i])
    print(lines[id])
    print("\n")

    for j in range(last, div):
        print(lines[j])
        last = div

[0, 0, 0, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 16, 17]

###
空空道人听如此说，思忖半晌，将一这《石头记》再检阅一遍
空空道人听如此说，思忖半晌，将一这《石头记》再检阅一遍，因见上面虽有些指奸责佞、贬恶诛邪之语，亦非伤时骂世之旨；及至君仁臣良、父慈子孝，凡伦常所关之处，皆是称功颂德，眷眷无穷，实非别书之可比。



###
因见上面虽有些指奸责佞、贬恶诛邪之语，亦非伤时骂世之旨
空空道人听如此说，思忖半晌，将一这《石头记》再检阅一遍，因见上面虽有些指奸责佞、贬恶诛邪之语，亦非伤时骂世之旨；及至君仁臣良、父慈子孝，凡伦常所关之处，皆是称功颂德，眷眷无穷，实非别书之可比。



###
及至君仁臣良、父慈子孝，凡伦常所关之处，皆是称功颂德，眷眷无穷
空空道人听如此说，思忖半晌，将一这《石头记》再检阅一遍，因见上面虽有些指奸责佞、贬恶诛邪之语，亦非伤时骂世之旨；及至君仁臣良、父慈子孝，凡伦常所关之处，皆是称功颂德，眷眷无穷，实非别书之可比。



###
实非别书之可比
空空道人听如此说，思忖半晌，将一这《石头记》再检阅一遍，因见上面虽有些指奸责佞、贬恶诛邪之语，亦非伤时骂世之旨；及至君仁臣良、父慈子孝，凡伦常所关之处，皆是称功颂德，眷眷无穷，实非别书之可比。



###
虽其中大旨谈情，亦不过实录其事，又非假拟妄称，一味淫邀艳约，私订偷盟之可比
虽其中大旨谈情，亦不过实录其事，又非假拟妄称，一味淫邀艳约，私订偷盟之可比。



###
因毫不干涉时世，方从头至尾抄录回来，问世传奇
因毫不干涉时世，方从头至尾抄录回来，问世传奇。



###
因空见色，由色生情，传情入色，自色悟空
因空见色，由色生情，传情入色，自色悟空，空空道人遂易名为情僧，改《石头记》为《情僧录》。



###
空空道人遂易名为情僧，改《石头记》为《情僧录》
因空见色，由色生情，传情入色，自色悟空，空空道人遂易名为情僧，改《石头记》为《情僧录》。



###
至?玉峰题曰《红楼梦》
至?玉峰题曰《红楼梦》。



###
东鲁孔梅溪则题曰《风月宝鉴》
东鲁孔梅溪则题曰《风月宝鉴》。



###
后因曹雪芹于悼红轩中，披阅十载，增删五次，纂成目录，分出章回
后因曹雪芹于悼红轩中，披阅十载，增删五次，

In [21]:
print(len(chunk_sum))
print(divs)
print(dia_texts)
print(seq)


# string_indices = [10, 11]
def jsonl_sorted(chunk_sum, divs, dia_texts, seq):

    combined_data = []
    combined_text = ""
    for index in sorted(seq + divs):
        # print(index)
        if index in seq:

            combined_data.append(
                {
                    "role": dialogues[seq.index(index)]["role"],
                    "text": dialogues[seq.index(index)]["dialogue"],
                    "if_scene": False,
                }
            )
            combined_text = (
                combined_text
                + dialogues[seq.index(index)]["role"]
                + ":"
                + dialogues[seq.index(index)]["dialogue"]
                + "\n"
            )
            seq[seq.index(index)] = -1
        if index in divs:
            combined_data.append(
                {
                    "role": "scene",
                    "text": chunk_sum[divs.index(index)],
                    "if_scene": True,
                }
            )
            combined_text = (
                combined_text + "scene" + ":" + chunk_sum[divs.index(index)] + "\n"
            )
            divs[divs.index(index)] = -1

    return combined_data, combined_text


combined_data, combined_text = jsonl_sorted(
    chunk_sum, divs.copy(), dia_texts, seq.copy()
)
print(combined_data)
# 打开文件，以写入模式（"w"）打开
with open(dialoge_file, "w", encoding="utf-8") as file:
    # 遍历数据列表中的每个字典
    for record in combined_data:
        # 将字典转换为JSON格式的字符串
        json_record = json.dumps(record, ensure_ascii=False)
        # 将转换后的JSON字符串写入文件，并添加换行符
        file.write(json_record + "\n")

8
[1, 2, 3, 4, 6, 8, 12, 17]
['空空道人听如此说，思忖半晌，将一这《石头记》再检阅一遍', '因见上面虽有些指奸责佞、贬恶诛邪之语，亦非伤时骂世之旨', '及至君仁臣良、父慈子孝，凡伦常所关之处，皆是称功颂德，眷眷无穷', '实非别书之可比', '虽其中大旨谈情，亦不过实录其事，又非假拟妄称，一味淫邀艳约，私订偷盟之可比', '因毫不干涉时世，方从头至尾抄录回来，问世传奇', '因空见色，由色生情，传情入色，自色悟空', '空空道人遂易名为情僧，改《石头记》为《情僧录》', '至?玉峰题曰《红楼梦》', '东鲁孔梅溪则题曰《风月宝鉴》', '后因曹雪芹于悼红轩中，披阅十载，增删五次，纂成目录，分出章回', '则题曰《金陵十二钗》，并题一绝云：满纸荒唐言，一把辛酸泪！都云作者痴，谁解其中味？', '至脂砚斋甲戌抄阅再评，仍用《石头记》', '出则既明，且看石上是何故事', '按那石上书云：当日地陷东南，这东南一隅有处曰姑苏，有城曰阊门者，最是红尘中一二等富贵风流之地', '这阊门外有个十里街，街内有个仁清巷，巷内有个古庙，因地方窄狭，人皆呼作葫芦庙', '庙旁住着一家乡宦，姓甄名费，字士隐', '嫡妻封氏，情性贤淑，深明礼义', '家中虽不甚富贵，然本地便也推他为望族了', '只因这甄士隐禀性恬淡，不以功名为念，每日只以观花修竹、酌酒吟诗为乐', '倒是神仙一流人品', '只是一件不足：如今年已半百，膝下无儿，只有一女，乳名英莲，年方三岁']
[0, 0, 0, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 16, 17]
[{'role': '空空道人', 'text': '空空道人听如此说，思忖半晌，将一这《石头记》再检阅一遍', 'if_scene': False}, {'role': '空空道人', 'text': '因见上面虽有些指奸责佞、贬恶诛邪之语，亦非伤时骂世之旨', 'if_scene': False}, {'role': '空空道人', 'text': '及至君仁臣良、父慈子孝，凡伦常所关之处，皆是称功颂德，眷眷无穷', 'if_scene': False}, {'role': '空空道人', 'text': '实非别书之可比', 'if_scene'

In [22]:
os.system(f"rm -rf {save_folder_path}")
os.makedirs(save_folder_path, exist_ok=True)

In [23]:
final_jsonl = []
final_txt = ""

for i in tqdm(
    range(1, len(chunk_text)), desc="Processing", total=len(chunk_text) - 1, unit="item"
):

    try:
        # story_name_en = 'shediaoyingxiongzhuan'
        raw_text = chunk_text[i]

        import os

        save_folder = f"./extract/{story_name_en}"

        dialoge_file = os.path.join(save_folder, f"{i}_dialogue.txt")
        summarzie_file = os.path.join(save_folder, f"{i}_sum.txt")

        chunk_sum = []
        unique_chunk_sum = []
        if os.path.exists(summarzie_file):
            with open(summarzie_file, "r", encoding="utf-8") as f:
                for line in f:
                    if line.strip().startswith("-"):
                        chunk_sum.append(line.strip()[1:].strip())
        if os.path.exists(dialoge_file):
            with open(dialoge_file, encoding="utf-8") as f:
                dialogues = []
                for line in f:
                    dialogue = json.loads(line)
                    dialogues.append(dialogue)

        unique_dialogue = []
        for item in dialogues:
            if item not in unique_dialogue:
                unique_dialogue.append(item)
        dia_texts = [data["dialogue"] for data in unique_dialogue]
        for item in chunk_sum:
            if item not in unique_chunk_sum:
                unique_chunk_sum.append(item)

        chunk_sum = unique_chunk_sum
        dialogues = unique_dialogue
        lines, starts, ends = divide_raw2lines(raw_text)
        # print(chunk_sum)
        # print(lines)
        score, divs = summary2line(chunk_sum, lines)  # summary匹配
        seq = dialogue2line(dia_texts, lines)  # 对话匹配
        combined_data, combined_text = jsonl_sorted(
            chunk_sum, divs.copy(), dia_texts, seq.copy()
        )
        # 如果需要保存每个chunk的，在此处保存
        final_jsonl.append(combined_data)
        final_txt = final_txt + combined_text + "\n"
    except:
        print("第" + str(i) + "个chunk出错")
        pass
with open(save_jsonl_path, "w", encoding="utf-8") as file:
    # 遍历数据列表中的每个字典
    for record in final_jsonl:
        # 将字典转换为JSON格式的字符串
        json_record = json.dumps(record, ensure_ascii=False)
        # 将转换后的JSON字符串写入文件，并添加换行符
        file.write(json_record + "\n")
with open(save_txt_path, "w", encoding="utf-8") as file:
    file.write(final_txt)

Processing:   2%|▏         | 16/958 [00:00<00:06, 154.61item/s]

第2个chunk出错


Processing:  30%|██▉       | 285/958 [00:02<00:04, 142.13item/s]

第260个chunk出错


Processing:  43%|████▎     | 412/958 [00:03<00:03, 138.99item/s]

第400个chunk出错


Processing:  76%|███████▌  | 727/958 [00:05<00:02, 92.00item/s] C:\Users\charles0618\AppData\Local\Temp\ipykernel_32624\1180376461.py:34: RuntimeWarning: invalid value encountered in divide
  recalls = freqs / np.array(query_chars_count)[:, None]
Processing: 100%|██████████| 958/958 [00:07<00:00, 122.94item/s]


# 4.小说chatbot抽取

In [24]:
# 参数设置

# 支持跨越多少行寻找目标角色，也即控制段内行间距不超过该值
max_find_lines = 10

max_token_num = 500



# 输入文件路径
input_jsonl_name = (
    f"./reorganized/{novel_name}/reorganized_{novel_name}.jsonl"
)

# 保存路径
savepath = f"./role/{novel_name}"
os.system(f"rm -rf {savepath}")
os.makedirs(savepath, exist_ok=True)
# zip_path = '/content/linghuchong_text.zip'

In [25]:
enc = tiktoken.get_encoding("cl100k_base")

data_in_chunk = []
with open(input_jsonl_name, encoding="utf8") as f:
    for line in f:
        data_in_chunk.append(json.loads(line))

data = []

for chunk in data_in_chunk:
    for d in chunk:
        data.append(d)

print(len(data))
for i, d in enumerate(data):
    if d["role"] == "scene":
        first_scene_id = i
        break
print(first_scene_id)

26970
3


In [26]:
def output_scene_chat_id(data, target_role_single):

    chat_ids = []

    # 先寻找所有出现角色的节点
    for i, d in enumerate(data):
        if d["role"] == target_role_single:
            chat_ids.append(i)

    previous_scene_ids = []

    # 对于每一个chat_ids，向前寻找scene的节点
    for chat_id in chat_ids:
        ans = first_scene_id
        for j in range(chat_id, first_scene_id, -1):
            if data[j]["role"] == "scene":
                ans = j
                break
        previous_scene_ids.append(ans)
    return chat_ids, previous_scene_ids

In [27]:
def divide_chats2chunks(chat_ids, previous_scene_ids):
    chat_ids_in_chunk = []
    current_chunk = []

    for chat_id in chat_ids:
        if not current_chunk:
            current_chunk.append(chat_id)
            continue

        if chat_id - current_chunk[-1] <= max_find_lines:
            current_chunk.append(chat_id)
        else:
            chat_ids_in_chunk.append(current_chunk)
            current_chunk = [chat_id]

    if current_chunk:
        chat_ids_in_chunk.append(current_chunk)

    # print(chat_ids_in_chunk[0])

    chat_id2previous_scene_id = {}

    for previous, chat_id in zip(previous_scene_ids, chat_ids):
        chat_id2previous_scene_id[chat_id] = previous
        if previous > 0:
            if data[previous - 1]["role"] != target_role:
                chat_id2previous_scene_id[chat_id] -= 1
    return chat_ids_in_chunk, chat_id2previous_scene_id


# chat_ids的分块， chat_id对应的旁白id

In [28]:
def count_token(my_str):
    return len(enc.encode(my_str))


def data2str(data):
    role = data["role"]
    if role in ["旁白", "", "scene", "Scene", "narrator", "Narrator"]:
        return "scene:" + data["text"]
    else:
        return role + ":「" + data["text"] + "」"

In [29]:
def id2texts(data, chat_ids_in_chunk, chat_id2previous_scene_id):
    line_token = [count_token(data2str(d)) for d in data]
    from ast import Break

    final_chunks = []

    print_count = 0

    appended_key = []

    for chunk in chat_ids_in_chunk:
        N = len(chunk)

        current_i = 0

        while current_i < N - 1:

            consider_chat_id = chunk[current_i]

            previous_scene_id = chat_id2previous_scene_id[consider_chat_id]

            # 保底
            withdraw_start = previous_scene_id
            withdraw_end = consider_chat_id

            current_count = sum(line_token[previous_scene_id : consider_chat_id + 1])
            while current_count < max_token_num and current_i < N - 1:
                consider_end = chunk[current_i + 1]
                consider_count = sum(line_token[previous_scene_id : consider_end + 1])
                if consider_count < max_token_num:
                    current_count = consider_count
                    withdraw_start = previous_scene_id
                    withdraw_end = consider_end
                    current_i += 1
                else:
                    break

            # print_count += 1

            # print(withdraw_start, end = ' ')
            # if print_count % 5 == 0:
            #     print()

            if withdraw_end + 1 not in appended_key:
                appended_key.append(withdraw_end + 1)
                chunk_str = ""
                for i in range(withdraw_start, withdraw_end + 1):
                    chunk_str += data2str(data[i]) + "\n"

                final_chunks.append(chunk_str)

            current_i += 1
    return appended_key, final_chunks

In [30]:
import zipfile
def save_chunk2zip(savepath, save_title, final_chunks):
    os.makedirs(savepath, exist_ok=True)
    for i in range(0, len(final_chunks)):
        my_str = final_chunks[i]
        with open(savepath + f"/text_{i}.txt", "w", encoding="utf-8") as f:
            f.write(my_str)
    zip_path = "./role/" + save_title + "_text.zip"
    with zipfile.ZipFile(zip_path, "w") as zipf:
        for filename in os.listdir(savepath):
            zipf.write(savepath + "/" + filename)

    print("Zipped folder saved to", zip_path)

In [31]:
for role_cur_name in target_role:
    chat_ids, previous_scene_ids = output_scene_chat_id(data, role_cur_name)
    chat_ids_in_chunk, chat_id2previous_scene_id = divide_chats2chunks(
        chat_ids, previous_scene_ids
    )
    appended_key, final_chunks = id2texts(
        data, chat_ids_in_chunk, chat_id2previous_scene_id
    )
    save_chunk2zip(
        savepath + "/" + role_cur_name, role_cur_name, final_chunks
    )  # 如果你想修改保存的zip名称，请修改本函数的第二个参数save_title

Zipped folder saved to ./role/黛玉_text.zip
Zipped folder saved to ./role/宝玉_text.zip
Zipped folder saved to ./role/宝钗_text.zip


# 5.抽取embedding
获得角色的jsonl资料后需要进行embedding构建支持RAG的知识库，这一步移到test文件里进行了